In [ ]:
import sagemaker
from sagemaker import get_execution_role
import json
import boto3

sess = sagemaker.Session(default_bucket='sagemaker-training-bucket-augustas')
role = get_execution_role()

bucket = 'sagemaker-training-bucket-augustas'
prefix = "blazingtext/supervised_model_training_data"

In [ ]:
!wget https://github.com/saurabh3949/Text-Classification-Datasets/raw/master/dbpedia_csv.tar.gz
!tar -xzvf dbpedia_csv.tar.gz
!head dbpedia_csv/train.csv -n 3

In [ ]:
index_to_label = {}
with open("training_data/classes.txt") as f:
    for i, label in enumerate(f.readlines()):
        index_to_label[str(i + 1)] = label.strip()
print(index_to_label)
